<a href="https://colab.research.google.com/github/AkritiGhosh/KrayenAssessment/blob/main/Streamlit_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Upload cnn model and 
from google.colab import files
files.upload()

In [ ]:
!mv '/content/gcmodel.h5'  '/content/model.h5'

In [ ]:
!pip install streamlit pyngrok

In [ ]:
%%writefile app.py

import streamlit as st
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageOps

PAGE_CONFIG = {"page_title":"Garbage Collection.io","page_icon":"chart_with_upwards_trend","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

def output(arr):
  classes = {1:'cardboard', 2:'glass',3:'metal',4:'paper',5:'plastic',6:'trash'}
  return classes[np.argmax(arr)]

def import_and_predict(image_data, model):
    size = (128,128)    
    image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
    image = np.asarray(image)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resize = (cv2.resize(img, dsize=(128, 128),interpolation=cv2.INTER_CUBIC))/255.
    img_reshape = img[np.newaxis,...]
    prediction = output(model.predict(img_reshape))
    return prediction

def load_model(path):
    model=tf.keras.models.load_model(path)
    return model

st.title("Krayen Assessment")
st.subheader('Garbage Collection')
with st.spinner('Model is being loaded..'):
  model=load_model('/content/model.h5')

file = st.file_uploader("Please upload an image", type=["jpg", "png"])
st.set_option('deprecation.showfileUploaderEncoding', False)
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    st.write('Prediction:')
    st.write(predictions)


Overwriting app.py


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np

PAGE_CONFIG = {"page_title":"IPL EDA.io","page_icon":"chart_with_upwards_trend","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

hr = '''

----

'''

def fillna(matches, deliveries, df_eda):
  df_eda.loc[df_eda['venue'] == 'Dubai International Cricket Stadium', 'city'] = 'Dubai'
  matches.loc[matches['venue'] == 'Dubai International Cricket Stadium', 'city'] = 'Dubai'
  df_eda['winner'].fillna('Tie', inplace=True)
  matches['winner'].fillna('Tie', inplace=True)
  df_eda['player_dismissed'].fillna('None', inplace=True)
  deliveries['player_dismissed'].fillna('None', inplace=True)
  df_eda['dismissal_kind'].fillna('None', inplace=True)
  deliveries['dismissal_kind'].fillna('None', inplace=True)
  df_eda = df_eda.fillna('NA')


st.title("Krayen Assessment")
st.subheader('IPL EDA')
matches = pd.read_csv('/content/matches.csv')
deliveries = pd.read_csv('/content/deliveries.csv')
df_eda = pd.merge(deliveries,matches, left_on='match_id', right_on='id', how ='outer')
del df_eda['id']
del df_eda['umpire3']
del df_eda['result']
del df_eda['fielder']
st.write('The merged data set -')
st.write(df_eda.head())
fillna(matches, deliveries, df_eda)
st.write('After filling empty values - ',df_eda.head())
st.markdown(hr)
seasons = np.sort(matches.season.unique())
st.write('Seasons-', seasons)
teams = list(set(matches.team1.unique()) | set(matches.team2.unique()))
players = list(set(df_eda.batsman.unique()) | set(df_eda.bowler.unique()))
st.write('Teams-', teams)
st.subheader('Team Performance')
st.write('Matches won by teams - ',pd.Series(matches['winner'].value_counts(), name = 'Matches Win'))
winners = list(matches['winner'].unique())
st.markdown(hr)
st.write("No of teams who didn't win any game -", len(list(set(teams) - set(winners))))
st.markdown(hr)
total_win = pd.Series(matches['winner'].value_counts(), name = 'Matches Win')
team_play = {}
for i in teams:
  team_play[i] = int(matches['team1'].value_counts()[i]) + int(matches['team2'].value_counts()[i])
total_play = pd.Series(team_play, name = 'Matches Play')
team_score = pd.merge(total_play, total_win, right_index = True,
               left_index = True)
team_score['Win%'] = (team_score['Matches Win'] / team_score['Matches Play']) * 100
st.write('Stats for each team',team_score.sort_values('Win%', ascending=False))

st.subheader('Season Performance')
season_winners = []
season = st.select_slider(
     'Select an year',
     options=seasons)
st.write('Season - ', season)
st.write('Season winner - ', matches[matches['season']==season].iloc[-1,:].winner) 
season_winners.append(matches[matches['season']==season].iloc[-1,:].winner)
st.write('Total number of matches - ', matches[matches['season']==season]['id'].count())
st.write('Matches won - ')
st.write(matches[matches['season']==season]['winner'].value_counts())
st.write(hr)


st.subheader('Dismissal')
dms_player = df_eda[df_eda['player_dismissed'] != 'None'][['player_dismissed','dismissal_kind']].value_counts().to_frame().reset_index()
dms_player['count'] = dms_player[0]
del dms_player[0]

st.write('',dms_player)

player_name = st.selectbox(
     'Select a player',
     players)
dms_player[dms_player['player_dismissed'] == player_name]
st.write(hr)

st.subheader('Bowlers')
bowlers = df_eda['bowler'].unique()
bowler_dismiss = df_eda[df_eda['player_dismissed'] != 'None'][['bowler']].value_counts().to_frame().reset_index()
bowler_dismiss['count'] = bowler_dismiss[0]
del bowler_dismiss[0]
bowler_name = st.selectbox('Number of outs by a bowler', bowlers)
st.write(bowler_dismiss[bowler_dismiss['bowler']==bowler_name])
st.write(hr)

st.subheader('Deliveries')
run_types = ['wide_runs','bye_runs','legbye_runs','noball_runs', 'penalty_runs', 'batsman_runs','extra_runs']
run_type = st.selectbox('Select a category of run', run_types)
rtb = df_eda.groupby(['season','bowler'])[run_type].sum().reset_index().sort_values(run_type, ascending=False)
st.write(rtb[:1])
st.write(hr)

st.write('Which batsman has highest number of runs in all seasons')
st.write(deliveries.groupby(['batsman'])['total_runs'].sum().reset_index().sort_values('total_runs', ascending=False)[:1])
st.write(hr)
st.write('How many runs by a player(batsman) in a season were batsman runs')
bat_total = df_eda.groupby(['season','batsman'])[['batsman_runs','total_runs']].sum().reset_index()
bat_total['bat_runs_percent'] = (bat_total['batsman_runs'])/(bat_total['total_runs'])
player_name1 = st.selectbox('Select a player', players,key='p1')
bat_total = bat_total[bat_total['batsman'] == player_name1]
st.write(bat_total)
season1 = st.select_slider('Select an year', options=seasons,key='s1')
bat_total = bat_total[bat_total['season'] == season1]
st.write(bat_total)

st.write(hr)
st.write('Which batsman has highest number of runs in per seasons, and in what season')
st.write(df_eda.groupby(['season','batsman'])['total_runs'].sum().reset_index().sort_values('total_runs', ascending=False)[:1])

st.write(hr)
st.write('In which match, the highest number of runs were scored in an over')
highest_run_over = deliveries.groupby(['match_id','inning','over','batsman'])['total_runs'].sum().reset_index().sort_values('total_runs', ascending=False)[:1]
st.write(highest_run_over) 
cols = st.multiselect(
     'Information about the match to be displayed',
     matches.columns)
matchID = highest_run_over['match_id']
matches[(matches['id']==int(matchID))][cols]


Overwriting app.py


In [ ]:
!ls

deliveries.csv	iplapp.py    ngrok-stable-linux-amd64.zip    sample_data
gcapp.py	matches.csv  ngrok-stable-linux-amd64.zip.1
gcmodel.h5	ngrok	     ngrok-stable-linux-amd64.zip.2


## Install ngrok

In [ ]:
!ngrok authtoken 2BWeZD23UX4URSa1orGf9pmX8Of_7zNhFiU6EfaDcJrDNnvN1

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-07-06 08:08:56--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  10.4MB/s    in 1.3s    

2022-07-06 08:08:58 (10.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://76cd-34-125-174-195.ngrok.io


Running application

In [ ]:
!streamlit run /content/app.py &>/dev/null&

In [ ]:
!pgrep streamlit

368
426
520
574
590
660
1121
1211
1317
1375
